In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, precision_score

In [ ]:
# Load the CSV file
df = pd.read_csv('data to apply Ramer Scheme')

In [ ]:
#Ramer Scheme
# Constants
g = 9.81  # m/s^2
cp = 1005  # J/(kg·K)
L = 2501000  # J/kg
Rd = 287.05  # J/(kg·K)
epsilon = 0.622
e_prime = 0.045  # Given constant

def kelvin_to_celsius(T):
    return T - 273.15  # Convert Kelvin to Celsius

def calculate_malr(T, q_s):
    T_celsius = kelvin_to_celsius(T)  # Convert temperature to Celsius for MALR calculation
    return (g * (1 + (L * q_s) / (Rd * T_celsius))) / (cp + (L**2 * q_s * epsilon) / (Rd * T_celsius**2))

def calculate_if_and_cloud_top(df):
    # Define pressure levels
    pressures = [400, 500, 650, 700, 750, 800, 850, 900, 950, 975]
    calculation_pressures = pressures[:7]  # Only consider up to 850 hPa for MALR and ELR calculations
    
    # Iterate through each row to calculate If and determine cloud-top layer
    results_if = []
    results_cloud_top = []
    for index, row in df.iterrows():
        # Find cloud-top layer
        cloud_top = None
        highest_rh = 0  # Variable to store the highest RH found
        highest_rh_layer = None  # Layer with the highest RH
        first_high_rh_layer = None  # First layer with RH > 90

        for i in range(len(calculation_pressures) - 1):
            p1, p2 = calculation_pressures[i], calculation_pressures[i+1]
            T1, T2 = kelvin_to_celsius(row[f'T{p1}']), kelvin_to_celsius(row[f'T{p2}'])
            q_s = row[f'{p1}SH']
            MALR = calculate_malr((T1 + T2) / 2, q_s)
            ELR = (T2 - T1) / (p2 - p1)
            
            # Track highest RH across all pressures
            if row[f'{p1}RH'] > highest_rh:
                highest_rh = row[f'{p1}RH']
                highest_rh_layer = p1

            # First check for RH > 90 and MALR >= ELR within the calculation range
            if row[f'{p1}RH'] > 90 and ELR <= MALR:
                cloud_top = p1
                break
            
            # Track the first layer with RH > 90 across all pressures
            if row[f'{p1}RH'] > 90 and first_high_rh_layer is None:
                first_high_rh_layer = p1

        # Second check if no cloud-top found with first criteria
        if cloud_top is None:
            cloud_top = first_high_rh_layer

        # Third check if no suitable layer found
        cloud_top = cloud_top if cloud_top is not None else highest_rh_layer
        results_cloud_top.append(cloud_top)
        
        # Calculate If from the cloud-top layer to 975 hPa
        if_value = 0
        if cloud_top:
            cloud_top_index = pressures.index(cloud_top)
            for i in range(cloud_top_index, len(pressures) - 1):
                p1 = pressures[i]
                p2 = pressures[i + 1]
                # Calculate dI
                dp = np.log(p1) - np.log(p2)
                dI = (0 - kelvin_to_celsius(row[f'{p1}WbT'])) * dp / (e_prime * row[f'{p1}RH'])
                if_value += dI
        
        results_if.append(if_value)
    
    return results_if, results_cloud_top

In [ ]:
# Calculate If and cloud-top and add them to the DataFrame
df['If'], df['Cloud_Top'] = calculate_if_and_cloud_top(df)

In [ ]:
# Apply Ramer Scheme conditions
def apply_ramer_scheme(row):
    cloud_top_wbt = kelvin_to_celsius(row[f'{int(row["Cloud_Top"])}WbT'])
    surface_wbt = kelvin_to_celsius(row['WbT'])
    wbt_975 = kelvin_to_celsius(row['975WbT'])
    if (cloud_top_wbt > -6.6 or wbt_975 > 0) and (-6.6 <= surface_wbt < 0) and row['If'] < 0.85:
        return 1
    return 0


df['Ramer'] = df.apply(apply_ramer_scheme, axis=1)

In [ ]:
#metrics for Ramer Scheme
df['FR'] = df['FR'].astype(int)
df['Ramer'] = df['Ramer'].astype(int)

# Calculate AUC
auc = roc_auc_score(df['FR'], df['Ramer'])
print(f"AUC: {auc}")

# Calculate Accuracy
accuracy = accuracy_score(df['FR'], df['Ramer'])
print(f"Accuracy: {accuracy}")

# Calculate Recall
recall = recall_score(df['FR'], df['Ramer'])
print(f"Recall: {recall}")

# Calculate Precision
precision = precision_score(df['FR'], df['Ramer'])
print(f"Precision: {precision}")

In [ ]:
#Save the results
df.to_csv('path to save Ramer Scheme', index=False)